In [ ]:
"""
HuGCDN Model Comparison for Pre-Apnea Detection
Compares: Neural Network (CNN), SVM, Random Forest

Tests different window sizes (T) and prediction horizons
to find optimal configuration for MAX30102 deployment
"""

import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (classification_report, confusion_matrix, 
                             accuracy_score, precision_recall_fscore_support,
                             roc_curve, auc)
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# TensorFlow for Neural Network
try:
    import tensorflow as tf
    from tensorflow.keras import layers, models
    from tensorflow.keras.callbacks import EarlyStopping
    TF_AVAILABLE = True
except:
    TF_AVAILABLE = False
    print("TensorFlow not available, will skip Neural Network")

# =============================================================================
# CONFIGURATION
# =============================================================================
BASE_DIR = Path(r"D:\School Stuff\Pradita University\SEM 5\MLDL\Pre-Apnea-Prediction")
DATA_DIR = BASE_DIR / "data"
HUGCDN_DIR = DATA_DIR / "HuGCDN2014-OXI"

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
if TF_AVAILABLE:
    tf.random.set_seed(RANDOM_STATE)

# Test different configurations
WINDOW_SIZES = [60, 120, 180, 200]  # seconds (1, 2, 3, 3.3 minutes)
PREDICTION_HORIZONS = [1, 2, 3]      # minutes ahead to predict

# Best config from testing (you'll update this)
T_BEST = 200
HORIZON_BEST = 1

# =============================================================================
# DATA LOADING FUNCTIONS
# =============================================================================

def load_labels(path: Path) -> np.ndarray:
    """Load per-minute labels"""
    d = loadmat(str(path))
    if "salida_man_1m" in d:
        raw = d["salida_man_1m"]
    elif "salida_man" in d:
        raw = d["salida_man"]
    else:
        keys = [k for k in d.keys() if not k.startswith("__")]
        raw = d[keys[0]]
    return np.ravel(raw).astype(int)


def extract_epoch_series(raw) -> list:
    """Convert MATLAB array to list of 1D arrays"""
    arr = np.array(raw)
    
    if arr.dtype == object:
        epochs = []
        for x in arr.ravel():
            x = np.array(x).astype(float).ravel()
            if x.size > 0:
                epochs.append(x)
        return epochs
    
    arr = np.squeeze(arr)
    if arr.ndim == 1:
        return [arr.astype(float)]
    else:
        return [arr[i, :].astype(float).ravel() for i in range(arr.shape[0])]


def load_sat_epochs(path: Path) -> list:
    """Load SpO2 epochs"""
    d = loadmat(str(path))
    for name in ["sat", "Sat", "SaO2", "satO2", "SaO2_1m"]:
        if name in d:
            raw = d[name]
            break
    else:
        keys = [k for k in d.keys() if not k.startswith("__")]
        raw = d[keys[0]]
    return extract_epoch_series(raw)


def resize_epoch(x: np.ndarray, target_len: int) -> np.ndarray:
    """Linearly resample to fixed length"""
    x = np.asarray(x, dtype=float).ravel()
    if x.size == 0:
        return np.zeros(target_len, dtype=float)
    if x.size == target_len:
        return x
    old_idx = np.linspace(0.0, 1.0, num=x.size)
    new_idx = np.linspace(0.0, 1.0, num=target_len)
    return np.interp(new_idx, old_idx, x)


def load_hugcdn_subject(subject_id: str, T: int = 200):
    """Load one subject with fixed-length epochs"""
    label_path = HUGCDN_DIR / "LABELS" / f"{subject_id}.mat"
    sat_path = HUGCDN_DIR / "SAT" / f"{subject_id}.mat"

    labels = load_labels(label_path)
    sat_ep = load_sat_epochs(sat_path)

    n_epochs = min(len(labels), len(sat_ep))
    if n_epochs == 0:
        raise ValueError(f"No usable epochs for {subject_id}")

    labels = labels[:n_epochs]
    sat_ep = sat_ep[:n_epochs]

    # Resample each epoch to length T
    sat_resampled = np.stack([resize_epoch(e, T) for e in sat_ep], axis=0)

    # Z-normalize per epoch
    mu = sat_resampled.mean(axis=1, keepdims=True)
    sigma = sat_resampled.std(axis=1, keepdims=True) + 1e-6
    sat_z = (sat_resampled - mu) / sigma

    # Reshape to (n_epochs, T, 1) for CNN
    X = sat_z[:, :, np.newaxis]
    y = labels.astype(int)

    return X, y


def load_all_hugcdn(T: int = 200):
    """Load all subjects"""
    label_dir = HUGCDN_DIR / "LABELS"
    subject_ids = sorted([f.stem for f in label_dir.glob("*.mat")])

    X_list = []
    y_list = []
    subj_idx = []

    print(f"Loading {len(subject_ids)} subjects with T={T}...")
    for sid in subject_ids:
        try:
            X_s, y_s = load_hugcdn_subject(sid, T=T)
            X_list.append(X_s)
            y_list.append(y_s)
            subj_idx.extend([sid] * len(y_s))
        except Exception as e:
            print(f"  Skipping {sid}: {e}")

    X = np.concatenate(X_list, axis=0)
    y = np.concatenate(y_list, axis=0)
    subject_idx = np.array(subj_idx)

    print(f"  Loaded: X={X.shape}, y={y.shape}")
    return X, y, subject_idx


def make_pre_apnea_labels(y, subject_idx, horizon=1):
    """Create pre-apnea labels (1 if apnea in next 'horizon' minutes)"""
    y = np.asarray(y).astype(int)
    pre_y = np.zeros_like(y)

    unique_subj = np.unique(subject_idx)
    for sid in unique_subj:
        mask = (subject_idx == sid)
        idxs = np.where(mask)[0]
        labels_sub = y[mask]

        for j, global_idx in enumerate(idxs):
            end_j = min(len(labels_sub), j + 1 + horizon)
            if np.any(labels_sub[j + 1:end_j] == 1):
                pre_y[global_idx] = 1

    return pre_y


# =============================================================================
# MODEL BUILDERS
# =============================================================================

def build_cnn_model(input_shape, model_size='small'):
    """Build CNN for TinyML deployment"""
    inputs = layers.Input(shape=input_shape)

    if model_size == 'tiny':
        # Minimal model for tight memory
        x = layers.Conv1D(8, kernel_size=5, activation="relu", padding="same")(inputs)
        x = layers.MaxPooling1D(pool_size=2)(x)
        x = layers.Conv1D(16, kernel_size=5, activation="relu", padding="same")(x)
        x = layers.GlobalAveragePooling1D()(x)
        x = layers.Dense(16, activation="relu")(x)
        
    elif model_size == 'small':
        # Balanced model (your original)
        x = layers.Conv1D(16, kernel_size=5, activation="relu", padding="same")(inputs)
        x = layers.MaxPooling1D(pool_size=2)(x)
        x = layers.Conv1D(32, kernel_size=5, activation="relu", padding="same")(x)
        x = layers.MaxPooling1D(pool_size=2)(x)
        x = layers.Conv1D(64, kernel_size=3, activation="relu", padding="same")(x)
        x = layers.GlobalAveragePooling1D()(x)
        x = layers.Dense(32, activation="relu")(x)
        
    else:  # 'medium'
        # Larger model for best accuracy
        x = layers.Conv1D(32, kernel_size=5, activation="relu", padding="same")(inputs)
        x = layers.MaxPooling1D(pool_size=2)(x)
        x = layers.Conv1D(64, kernel_size=5, activation="relu", padding="same")(x)
        x = layers.MaxPooling1D(pool_size=2)(x)
        x = layers.Conv1D(128, kernel_size=3, activation="relu", padding="same")(x)
        x = layers.GlobalAveragePooling1D()(x)
        x = layers.Dense(64, activation="relu")(x)
        x = layers.Dropout(0.3)(x)
        x = layers.Dense(32, activation="relu")(x)

    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )
    return model


# =============================================================================
# EVALUATION FUNCTION
# =============================================================================

def evaluate_model(y_true, y_pred, y_pred_proba=None, model_name="Model"):
    """Comprehensive evaluation metrics"""
    
    results = {}
    
    # Basic metrics
    results['accuracy'] = accuracy_score(y_true, y_pred)
    prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary', zero_division=0)
    results['precision'] = prec
    results['recall'] = rec
    results['f1'] = f1
    
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    results['tn'] = tn
    results['fp'] = fp
    results['fn'] = fn
    results['tp'] = tp
    results['specificity'] = tn / (tn + fp) if (tn + fp) > 0 else 0
    
    # ROC AUC if probabilities available
    if y_pred_proba is not None:
        fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
        results['auc'] = auc(fpr, tpr)
    else:
        results['auc'] = None
    
    # Print results
    print(f"\n{'='*60}")
    print(f"{model_name} Results")
    print(f"{'='*60}")
    print(f"Accuracy:    {results['accuracy']*100:.2f}%")
    print(f"Precision:   {results['precision']*100:.2f}%")
    print(f"Recall:      {results['recall']*100:.2f}%")
    print(f"F1-Score:    {results['f1']*100:.2f}%")
    print(f"Specificity: {results['specificity']*100:.2f}%")
    if results['auc']:
        print(f"AUC:         {results['auc']:.4f}")
    print(f"\nConfusion Matrix:")
    print(f"  TN: {tn:4d}  FP: {fp:4d}")
    print(f"  FN: {fn:4d}  TP: {tp:4d}")
    
    return results


# =============================================================================
# MAIN COMPARISON
# =============================================================================

def main():
    print("="*80)
    print("HuGCDN Pre-Apnea Model Comparison")
    print("Neural Network (CNN) vs SVM vs Random Forest")
    print("="*80)
    print()
    
    # Load data with best configuration
    print(f"Loading data with T={T_BEST}, HORIZON={HORIZON_BEST}...")
    X, y_apnea, subject_idx = load_all_hugcdn(T=T_BEST)
    
    # Create pre-apnea labels
    y_pre = make_pre_apnea_labels(y_apnea, subject_idx, horizon=HORIZON_BEST)
    
    print(f"\nDataset Statistics:")
    print(f"  Total samples: {len(y_pre)}")
    print(f"  Apnea events: {np.sum(y_apnea)} ({np.sum(y_apnea)/len(y_apnea)*100:.1f}%)")
    print(f"  Pre-apnea: {np.sum(y_pre)} ({np.sum(y_pre)/len(y_pre)*100:.1f}%)")
    print()
    
    # Shuffle and split
    X_shuf, y_shuf = shuffle(X, y_pre, random_state=RANDOM_STATE)
    
    X_train, X_temp, y_train, y_temp = train_test_split(
        X_shuf, y_shuf, test_size=0.3, random_state=RANDOM_STATE, stratify=y_shuf
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, random_state=RANDOM_STATE, stratify=y_temp
    )
    
    print(f"Split: Train={len(X_train)}, Val={len(X_val)}, Test={len(X_test)}")
    print()
    
    # Calculate class weights
    binc = np.bincount(y_train)
    if len(binc) == 1:
        class_weight = {0: 1.0}
    else:
        total = float(binc.sum())
        class_weight = {
            0: total / (2.0 * binc[0]),
            1: total / (2.0 * binc[1]),
        }
    print(f"Class weights: {class_weight}")
    print()
    
    # Storage for results
    all_results = []
    
    # =========================================================================
    # MODEL 1: NEURAL NETWORK (CNN)
    # =========================================================================
    if TF_AVAILABLE:
        print("\n" + "="*80)
        print("1. NEURAL NETWORK (CNN) - Multiple Sizes")
        print("="*80)
        
        for model_size in ['tiny', 'small', 'medium']:
            print(f"\n--- CNN ({model_size.upper()}) ---")
            
            model = build_cnn_model(input_shape=(T_BEST, 1), model_size=model_size)
            
            # Count parameters
            total_params = model.count_params()
            print(f"Total parameters: {total_params:,}")
            
            # Estimate model size
            model_size_kb = (total_params * 4) / 1024  # 4 bytes per float32
            print(f"Estimated model size: {model_size_kb:.1f} KB (float32)")
            print(f"Quantized (int8): ~{model_size_kb/4:.1f} KB")
            
            # Train
            early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
            
            history = model.fit(
                X_train, y_train,
                validation_data=(X_val, y_val),
                epochs=50,
                batch_size=64,
                class_weight=class_weight,
                callbacks=[early_stop],
                verbose=0
            )
            
            # Evaluate
            y_pred_proba = model.predict(X_test, verbose=0).ravel()
            y_pred = (y_pred_proba >= 0.5).astype(int)
            
            results = evaluate_model(y_test, y_pred, y_pred_proba, 
                                    model_name=f"CNN ({model_size})")
            results['model'] = f'CNN_{model_size}'
            results['params'] = total_params
            results['size_kb'] = model_size_kb
            all_results.append(results)
    
    # =========================================================================
    # MODEL 2: SVM
    # =========================================================================
    print("\n" + "="*80)
    print("2. SUPPORT VECTOR MACHINE (SVM)")
    print("="*80)
    
    # Flatten X for SVM (needs 2D input)
    X_train_flat = X_train.reshape(X_train.shape[0], -1)
    X_test_flat = X_test.reshape(X_test.shape[0], -1)
    
    # Standardize
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_flat)
    X_test_scaled = scaler.transform(X_test_flat)
    
    # Try different kernels
    for kernel in ['rbf', 'linear']:
        print(f"\n--- SVM ({kernel.upper()} kernel) ---")
        
        svm_model = SVC(
            kernel=kernel,
            C=1.0,
            gamma='scale' if kernel == 'rbf' else 'auto',
            random_state=RANDOM_STATE,
            probability=True,
            class_weight='balanced'
        )
        
        svm_model.fit(X_train_scaled, y_train)
        
        y_pred = svm_model.predict(X_test_scaled)
        y_pred_proba = svm_model.predict_proba(X_test_scaled)[:, 1]
        
        results = evaluate_model(y_test, y_pred, y_pred_proba, 
                                model_name=f"SVM ({kernel})")
        results['model'] = f'SVM_{kernel}'
        results['params'] = len(svm_model.support_vectors_)
        results['size_kb'] = None  # Hard to estimate
        all_results.append(results)
    
    # =========================================================================
    # MODEL 3: RANDOM FOREST
    # =========================================================================
    print("\n" + "="*80)
    print("3. RANDOM FOREST")
    print("="*80)
    
    # Try different sizes
    for n_trees in [10, 50, 100]:
        print(f"\n--- Random Forest ({n_trees} trees) ---")
        
        rf_model = RandomForestClassifier(
            n_estimators=n_trees,
            max_depth=20,
            min_samples_split=10,
            min_samples_leaf=5,
            random_state=RANDOM_STATE,
            class_weight='balanced',
            n_jobs=-1
        )
        
        rf_model.fit(X_train_flat, y_train)
        
        y_pred = rf_model.predict(X_test_flat)
        y_pred_proba = rf_model.predict_proba(X_test_flat)[:, 1]
        
        results = evaluate_model(y_test, y_pred, y_pred_proba, 
                                model_name=f"Random Forest ({n_trees} trees)")
        results['model'] = f'RF_{n_trees}'
        results['params'] = None  # Complex to estimate
        results['size_kb'] = None
        all_results.append(results)
    
    # =========================================================================
    # COMPARISON VISUALIZATION
    # =========================================================================
    results_df = pd.DataFrame(all_results)
    
    print("\n\n" + "="*80)
    print("COMPREHENSIVE COMPARISON")
    print("="*80)
    print(results_df[['model', 'accuracy', 'precision', 'recall', 'f1', 'auc']].to_string(index=False))
    
    # Plot comparison
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Accuracy comparison
    ax1 = axes[0, 0]
    models = results_df['model']
    accuracies = results_df['accuracy'] * 100
    
    colors = ['steelblue' if 'CNN' in m else 'forestgreen' if 'SVM' in m else 'coral' 
              for m in models]
    
    bars = ax1.barh(range(len(models)), accuracies, color=colors, edgecolor='black', linewidth=1.5)
    ax1.set_yticks(range(len(models)))
    ax1.set_yticklabels(models, fontsize=9)
    ax1.set_xlabel('Accuracy (%)', fontsize=11, fontweight='bold')
    ax1.set_title('Model Accuracy Comparison', fontsize=12, fontweight='bold')
    ax1.grid(True, alpha=0.3, axis='x')
    
    for i, (bar, acc) in enumerate(zip(bars, accuracies)):
        ax1.text(acc + 1, i, f'{acc:.1f}%', va='center', fontweight='bold', fontsize=9)
    
    # 2. F1-Score comparison
    ax2 = axes[0, 1]
    f1_scores = results_df['f1'] * 100
    
    bars = ax2.barh(range(len(models)), f1_scores, color=colors, edgecolor='black', linewidth=1.5)
    ax2.set_yticks(range(len(models)))
    ax2.set_yticklabels(models, fontsize=9)
    ax2.set_xlabel('F1-Score (%)', fontsize=11, fontweight='bold')
    ax2.set_title('F1-Score Comparison', fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3, axis='x')
    
    for i, (bar, f1) in enumerate(zip(bars, f1_scores)):
        ax2.text(f1 + 1, i, f'{f1:.1f}%', va='center', fontweight='bold', fontsize=9)
    
    # 3. Precision vs Recall
    ax3 = axes[1, 0]
    for idx, row in results_df.iterrows():
        color = 'steelblue' if 'CNN' in row['model'] else 'forestgreen' if 'SVM' in row['model'] else 'coral'
        ax3.scatter(row['recall']*100, row['precision']*100, s=200, 
                   c=color, edgecolors='black', linewidths=2, alpha=0.7)
        ax3.annotate(row['model'], (row['recall']*100, row['precision']*100), 
                    fontsize=8, ha='right', va='bottom')
    
    ax3.set_xlabel('Recall (%)', fontsize=11, fontweight='bold')
    ax3.set_ylabel('Precision (%)', fontsize=11, fontweight='bold')
    ax3.set_title('Precision vs Recall', fontsize=12, fontweight='bold')
    ax3.grid(True, alpha=0.3)
    ax3.plot([0, 100], [0, 100], 'k--', alpha=0.3, label='Perfect balance')
    ax3.legend()
    
    # 4. Confusion matrix for best model
    ax4 = axes[1, 1]
    best_idx = results_df['f1'].idxmax()
    best = results_df.iloc[best_idx]
    
    cm = np.array([[best['tn'], best['fp']], [best['fn'], best['tp']]])
    
    im = ax4.imshow(cm, cmap='Blues', alpha=0.8)
    ax4.set_xticks([0, 1])
    ax4.set_yticks([0, 1])
    ax4.set_xticklabels(['Normal', 'Pre-Apnea'], fontsize=10)
    ax4.set_yticklabels(['Normal', 'Pre-Apnea'], fontsize=10)
    ax4.set_xlabel('Predicted', fontsize=11, fontweight='bold')
    ax4.set_ylabel('Actual', fontsize=11, fontweight='bold')
    ax4.set_title(f'Best Model: {best["model"]}\nF1={best["f1"]*100:.1f}%', 
                 fontsize=12, fontweight='bold')
    
    for i in range(2):
        for j in range(2):
            ax4.text(j, i, int(cm[i, j]), ha="center", va="center",
                    color="white" if cm[i, j] > cm.max()/2 else "black",
                    fontsize=16, fontweight='bold')
    
    plt.colorbar(im, ax=ax4)
    
    plt.tight_layout()
    
    # =========================================================================
    # FINAL RECOMMENDATIONS
    # =========================================================================
    print("\n\n" + "="*80)
    print("🎯 RECOMMENDATIONS FOR ESP32 DEPLOYMENT")
    print("="*80)
    
    best = results_df.loc[results_df['f1'].idxmax()]
    
    print(f"\n🏆 BEST OVERALL MODEL: {best['model']}")
    print(f"   Accuracy:  {best['accuracy']*100:.2f}%")
    print(f"   F1-Score:  {best['f1']*100:.2f}%")
    print(f"   Recall:    {best['recall']*100:.2f}% (detection rate)")
    print(f,"   Precision: {best['precision']*100:.2f}% (false alarm rate)")
    if best['auc']:
        print(f"   AUC:       {best['auc']:.4f}")
    
    # Best per category
    cnn_models = results_df[results_df['model'].str.contains('CNN')]
    if len(cnn_models) > 0:
        best_cnn = cnn_models.loc[cnn_models['f1'].idxmax()]
        print(f"\n📱 BEST FOR ESP32 (CNN): {best_cnn['model']}")
        print(f"   F1-Score: {best_cnn['f1']*100:.2f}%")
        print(f"   Model size: {best_cnn['size_kb']:.1f} KB (float32), ~{best_cnn['size_kb']/4:.1f} KB (int8)")
        print(f"   Parameters: {best_cnn['params']:,}")
        
        if best_cnn['size_kb'] < 100:
            print(f"   ✅ FITS EASILY on ESP32 (520KB SRAM available)")
        else:
            print(f"   ⚠️ May be tight on ESP32, consider quantization")
    
    svm_models = results_df[results_df['model'].str.contains('SVM')]
    if len(svm_models) > 0:
        best_svm = svm_models.loc[svm_models['f1'].idxmax()]
        print(f"\n🔵 BEST SVM: {best_svm['model']}")
        print(f"   F1-Score: {best_svm['f1']*100:.2f}%")
        print(f"   Note: SVM harder to deploy on ESP32 (needs CMSIS-NN or manual implementation)")
    
    rf_models = results_df[results_df['model'].str.contains('RF')]
    if len(rf_models) > 0:
        best_rf = rf_models.loc[rf_models['f1'].idxmax()]
        print(f"\n🌲 BEST RANDOM FOREST: {best_rf['model']}")
        print(f"   F1-Score: {best_rf['f1']*100:.2f}%")
        print(f"   Note: RF can be deployed using EloquentTinyML or manual implementation")
    
    print("\n" + "="*80)
    print("NEXT STEPS:")
    print("="*80)
    print("""
1. If CNN performs best:
   → Use TensorFlow Lite for Microcontrollers
   → Quantize to INT8 for smaller size
   → Already have ESP32 code template!
   
2. If SVM performs best:
   → Consider emlearn library for ESP32
   → Or implement decision function manually
   
3. If Random Forest performs best:
   → Use EloquentTinyML library
   → Or export as C code
   
4. Model optimization:
   → Try different window sizes (T)
   → Adjust prediction horizon
   → Feature engineering (add desaturation features)
    """)
    
    plt.show()
    
    # Save results
    results_df.to_csv(BASE_DIR / "model_comparison_results.csv", index=False)
    print(f"\n✓ Results saved to: {BASE_DIR / 'model_comparison_results.csv'}")


if __name__ == "__main__":
    main()

TensorFlow not available, will skip Neural Network
HuGCDN Pre-Apnea Model Comparison
Neural Network (CNN) vs SVM vs Random Forest

Loading data with T=200, HORIZON=1...
Loading 83 subjects with T=200...
  Loaded: X=(33367, 200, 1), y=(33367,)

Dataset Statistics:
  Total samples: 33367
  Apnea events: 7706 (23.1%)
  Pre-apnea: 7706 (23.1%)

Split: Train=23356, Val=5005, Test=5006

Class weights: {0: 0.6501503173365994, 1: 2.164998146088246}


2. SUPPORT VECTOR MACHINE (SVM)

--- SVM (RBF kernel) ---

SVM (rbf) Results
Accuracy:    80.38%
Precision:   55.88%
Recall:      71.54%
F1-Score:    62.75%
Specificity: 83.04%
AUC:         0.8464

Confusion Matrix:
  TN: 3197  FP:  653
  FN:  329  TP:  827

--- SVM (LINEAR kernel) ---
